In [568]:
import os
import time
import re
import pdfplumber
from decimal import Decimal

In [1]:
!pwd

/Users/binzhou/PycharmProjects/testsh/files


# 1. 用pdfplumber解析文本

In [12]:
# 识别流程一 detect_catalog_pagination
import re
def detect_catalog_pagination(pdf):
        catalog_type = 'clean'
        # 定位一、二
        for i, page in enumerate(pdf.pages):
                if [i for i in page.extract_text().split('\n') if i.strip().replace(' ', '') in ('目录', '条款目录')]:
                    return i, catalog_type
                if [i for i in page.extract_text().split('\n') if ''.join(re.findall(u"([\u4e00-\u9fa5])",i)) in ('目录', '条款目录')]:
                    catalog_type = 'impurity'
                    return i, catalog_type
        # 定位三
        for i, page in enumerate(pdf.pages):
            if '条款目录' in page.extract_text().strip().replace(' ',''):
                catalog_type = 'impurity'
                return i, catalog_type
        return -1, catalog_type

## 1.1 form_1 目录比较清晰，按顺序下来，没有左右的情况）（左右情况见富德人寿） clean的情况

In [803]:
feePDF = 'data/1-太平荣耀钻账户终身寿险（万能型）条款.pdf'
pdf = pdfplumber.open(feePDF)
catalog_idx, catalog_type = detect_catalog_pagination(pdf)
catalog_idx, catalog_type

(1, 'clean')

In [804]:
for i, page in enumerate(pdf.pages):
    a = page.extract_text()
    if i == 1:
        break

In [805]:
def catalog_ext_form1(page_text):
    flag = False
    res = []
    page_text_clean = [i.strip() for i in page_text.split('\n') if i.strip() and not re.findall(r'[\u4e00-\u9fa5]+\d+[\u4e00-\u9fa5]*', i) and \
     (i.strip().replace(' ', '') != '条款目录' and i.strip().replace(' ', '') != '目录')]
    res = [re.sub(r'\.\.+', '', i).split() for i in page_text_clean]
    return res

In [806]:
catalog_ext_form1(a)

[['第一部分', '您（投保人）与我们（太平人寿保险有限公司）的合同', '4'],
 ['第一条', '保险合同的构成', '4'],
 ['第二条', '投保范围', '4'],
 ['第三条', '保险合同成立与生效', '4'],
 ['第四条', '保险期间', '4'],
 ['第二部分', '我们提供哪些保障利益', '4'],
 ['第五条', '基本保险金额及有效保险金额', '4'],
 ['第六条', '保险责任', '4'],
 ['第七条', '责任免除', '5'],
 ['第三部分', '如何支付保险费', '5'],
 ['第八条', '保险费的支付', '5'],
 ['第九条', '风险保险费', '5'],
 ['第十条', '宽限期', '6'],
 ['第十一条', '相关费用', '6'],
 ['第十二条', '持续奖励', '6'],
 ['第四部分', '我们的账户是如何运作的', '6'],
 ['第十三条', '账户运作', '6'],
 ['第十四条', '保单账户价值', '6'],
 ['第十五条', '保证利率', '7'],
 ['第十六条', '结算利率', '7'],
 ['第十七条', '保单利息', '7'],
 ['第五部分', '如何申请保险金', '7'],
 ['第十八条', '受益人', '7'],
 ['第十九条', '保险事故通知', '8'],
 ['第二十条', '诉讼时效', '8'],
 ['第二十一条', '保险金申请', '8'],
 ['第二十二条', '保险金给付', '8'],
 ['第二十三条', '司法鉴定', '8'],
 ['第二十四条', '失踪处理', '8'],
 ['第六部分', '您还享有哪些权益', '9'],
 ['第二十五条', '保单贷款', '9'],
 ['第二十六条', '保单账户价值的部分提取权', '9'],
 ['第二十七条', '合同内容的变更权', '9'],
 ['第二十八条', '合同效力的中止与恢复', '9'],
 ['第二十九条', '犹豫期', '10'],
 ['第三十条', '您解除合同的手续及风险', '10'],
 ['第七部分', '您必须了解的其他事项', '10'],
 ['第三十一条', '明

## 1.2 form_2 目录竖的下来，可能有两三列（比如天安人寿）
### 考虑clean和impurity两种情况

In [728]:
# feePDF = 'data/华康C款团体重大疾病保险条款.pdf'
feePDF = 'data/1-太平荣耀钻账户终身寿险（万能型）条款.pdf'
feePDF = 'data/天安人寿爱守护（至尊保）终身重大疾病保险条款.pdf'
pdf = pdfplumber.open(feePDF)
catalog_idx, catalog_type = detect_catalog_pagination(pdf)
catalog_idx, catalog_type

(0, 'clean')

In [729]:
for i, page in enumerate(pdf.pages):
    a = page.extract_text()
    if i == 0:
        break

In [292]:
'''
从条款目录开始
过滤掉目录到最后杂质
'''

'\n从条款目录开始\n过滤掉目录到最后杂质\n'

In [478]:
# 版本一
def catalog_ext_form2(page_text):
    flag = False
    res = []
    for s in page_text.split('\n'):
        if s.strip().replace(' ', '') in ('目录', '条款目录'):
            flag = True
            continue
        if flag:
            lst = re.findall(r'(?:\d\.\d*[\u4e00-\u9fa5]+[、，!！&%@#$]*[\u4e00-\u9fa5]*)', s.strip().replace('．', '.').replace(' ', ''))
            if not lst:
                continue
            lst = [re.findall(r'(\d\.\d*)([\u4e00-\u9fa5]+[、，!！&%@#$]*[\u4e00-\u9fa5]*)', i)[0] for i in lst]
            res += lst
    return sorted(res, key=lambda x : (float(x[0].split('.')[0]), len(x[0]), float(x[0])))

In [245]:
# 版本二
def catalog_ext_form2(page_text):
    flag = False
    res = []
    _d = dict()
    _d_i = 97
    for s in page_text.split('\n'):
        if s.strip().replace(' ', '') in ('目录', '条款目录'):
            flag = True
            continue
        if flag:
            l1 = re.findall(r'\d+种', s)
            if l1:
                s = s.replace(l1[0], '龘龘'+chr(_d_i))
                _d.setdefault('龘龘'+chr(_d_i), l1[0])
                _d_i += 1
            lst = re.findall(r'(?:\d+\.\d*[\u4e00-\u9fa5]+[a-z]*\d*[\u4e00-\u9fa5]+[、，!！&%@#$]*[\u4e00-\u9fa5]*)', s.strip().replace('．', '.').replace(' ', ''))
            if not lst:
                continue
            lst = [re.findall(r'(\d+\.\d*)([\u4e00-\u9fa5]+[a-z]*\d*[\u4e00-\u9fa5]+[、，!！&%@#$]*[\u4e00-\u9fa5]*)', i)[0] for i in lst]
            for k, v in lst:
                l2 = re.findall(r'龘龘[a-z]+', v)
                if l2:
                    v = v.replace(l2[0], _d[l2[0]])
                res.append((k, v))
    return sorted(res, key=lambda x : (float(x[0].split('.')[0]), len(x[0]), float(x[0])))

In [236]:
# 版本三
def catalog_ext_form2(page_text):
    flag = False
    res = []
    _d = dict()
    _d_i = 97
    for s in page_text.split('\n'):
        if s.strip().replace(' ', '') in ('目录', '条款目录'):
            flag = True
            continue
        if flag:
            # 85种xxx的情况
            l1 = re.findall(r'\d+种', s)
            if l1:
                s = s.replace(l1[0], '龘龘'+chr(_d_i))
                _d.setdefault('龘龘'+chr(_d_i), l1[0])
                _d_i += 1

            # 目录数标没有点的情况，比如 【5 保险费的交纳  10其他需要关注的事项】
            l3 = re.findall(r'(?:^\d+(?!\.)\s+)|(?:\s+\d+(?!\d+)(?!\.))(?!种)', s)
            if l3:
                for i in l3:
                    s = s.replace(i, ' '+i.strip()+'. ')
            #
            lst = re.findall(r'(?:\d+\.\d*[\u4e00-\u9fa5]+[a-z]*\d*[\u4e00-\u9fa5]+[、，!！&%@#$]*[\u4e00-\u9fa5]*)', s.strip().replace('．', '.').replace(' ', ''))
            if not lst:
                continue
            lst = [re.findall(r'(\d+\.\d*)([\u4e00-\u9fa5]+[a-z]*\d*[\u4e00-\u9fa5]+[、，!！&%@#$]*[\u4e00-\u9fa5]*)', i)[0] for i in lst]
            for k, v in lst:
                l2 = re.findall(r'龘龘[a-z]+', v)
                if l2:
                    v = v.replace(l2[0], _d[l2[0]])
                res.append((k, v))
    return sorted(res, key=lambda x : (float(x[0].split('.')[0]), len(x[0]), float(x[0])))

In [730]:
# small tools
def xmerge(a, b):
    alen, blen = len(a), len(b)
    mlen = min(alen, blen)
    for i in range(mlen):
        yield a[i]
        yield b[i]

    if alen > blen:
        for i in range(mlen, alen):
            yield a[i]
    else:
        for i in range(mlen, blen):
            yield b[i]

In [731]:
# 版本四

def catalog_ext_form2(page_text, mode):
    flag = False
    res = []
    _d = dict()
    _d_i = 97
    for s in page_text.split('\n'):
        if not flag:
            if mode == 'clean' and s.strip().replace(' ', '') in ('目录', '条款目录'):
                flag = True
                continue
            if mode == 'impurity' and ('条款目录' in s.strip().replace(' ', '') or '目录' in s.strip().replace(' ', '')):
                flag = True
                continue
        if flag:
            # 85种xxx的情况
            l1 = re.findall(r'\d+种', s)
            if l1:
                s = s.replace(l1[0], '龘龘'+chr(_d_i))
                _d.setdefault('龘龘'+chr(_d_i), l1[0])
                _d_i += 1
            # 先把字符串合并，即strip然后去掉所有空格
            s = s.strip().replace('．', '.').replace(' ', '')
            # 目录数标没有点的情况，比如 【5保险费的交纳10其他需要关注的事项】
            l3 = re.findall(r'\d+\.*\d*', s)
            if l3:
                l3_1 = re.split(r'\d+\.*\d*', s)
                l3 = [i+'.' if '.' not in i else i for i in l3]
                s = ''.join([i for i in xmerge(l3_1, l3)])
            #
            lst = re.findall(r'(?:\d+\.\d*[\u4e00-\u9fa5]+[a-z]*\d*[\u4e00-\u9fa5]+[、，!！&%@#$]*[\u4e00-\u9fa5]*)', s)
            if not lst:
                continue
            lst = [re.findall(r'(\d+\.\d*)([\u4e00-\u9fa5]+[a-z]*\d*[\u4e00-\u9fa5]+[、，!！&%@#$]*[\u4e00-\u9fa5]*)', i)[0] for i in lst]
            for k, v in lst:
                l2 = re.findall(r'龘龘[a-z]+', v)
                if l2:
                    v = v.replace(l2[0], _d[l2[0]])
                res.append((k, v))
    return sorted(res, key=lambda x : (float(x[0].split('.')[0]), len(x[0]), float(x[0])))

In [732]:
# mode = 'impurity'
mode = 'clean'
catalog_ext_form2(a, mode)

[('1.', '您与我们的合同'),
 ('1.1', '投保范围'),
 ('1.2', '合同构成'),
 ('1.3', '合同成立与生效'),
 ('1.4', '犹豫期'),
 ('1.5', '合同内容变更'),
 ('1.6', '您解除合同的手续'),
 ('1.7', '合同终止'),
 ('2.', '我们提供的保障'),
 ('2.1', '基本保险金额'),
 ('2.2', '保险期间'),
 ('2.3', '保险责任'),
 ('2.4', '责任免除'),
 ('3.', '您的权利和义务'),
 ('3.1', '保险费的交纳'),
 ('3.2', '续期保险费的'),
 ('3.3', '合同效力中止'),
 ('3.4', '合同效力恢复'),
 ('3.5', '减保'),
 ('3.6', '保单质押贷款'),
 ('3.7', '年金转换选择'),
 ('3.8', '保险费自动垫交'),
 ('4.', '保险金的申请与给付'),
 ('4.1', '保险金受益人的指定和变更'),
 ('4.2', '保险事故通知'),
 ('4.3', '保险金及豁免保险费的申请'),
 ('4.4', '保险金的给付'),
 ('4.5', '未还款项的扣除'),
 ('4.6', '诉讼时效'),
 ('5.', '其他需要关注的事项'),
 ('5.1', '明确说明与如实告知'),
 ('5.2', '本公司合同解除权的限制'),
 ('5.3', '年龄确定与错误处理'),
 ('5.4', '地址变更'),
 ('5.5', '失踪处理'),
 ('5.6', '争议处理'),
 ('6.', '释义'),
 ('6.1', '周岁'),
 ('6.2', '现金价值'),
 ('6.3', '认可医院'),
 ('6.4', '专科医生'),
 ('6.5', '初次发生'),
 ('6.6', '本合同所指的轻症疾病'),
 ('6.7', '意外伤害'),
 ('6.8', '本合同所指的中症疾病'),
 ('6.9', '本合同所指的重大疾病'),
 ('6.10', '住院'),
 ('6.11', '实际住院天数'),
 ('6.12', '保单年度'),
 ('6.13', '重大疾病分组'),
 ('6

In [231]:
def analyzePDF_step1_lineInfo(pdfPath):
    '''
    对pdf进行初步解析，将解析结果存入字典
    :param pdfPath:
    :return:
    pdfDic = {int pageIndex: # pageIndex
              {
                  'lineDic': {},  # {y0:{'element':list,'content':str}}
                  'DiseaseRegion': {}  # {x0:float,y0:float,x1:float,y1:float,bboxList[(4-tuple),...]}
              }
              }
    '''

    pdfDic = {}
    with pdfplumber.open(pdfPath) as pdf:
        for i, page in enumerate(pdf.pages):
            pageNo = i+1
            pdfDic[pageNo] = {}
            #pdfDic[pageNo] = {'pdfplumberPage':page}
            chars = page.chars
            # 1.字符组合成行：所有纵坐标一样的char形成line
            lineDic = {}
            for char in chars:
                # 把每个char归到各个行中
                y0 = float(char['y0'])
                hit = 0
                for key in lineDic.keys():
                    if abs(y0 - key) < float(char['height']) * 0.3:
                        hit += 1
                        lineDic[key]['element'].append(char)
                if hit == 0:
                    lineDic[y0] = {'element': []}
                    lineDic[y0]['element'].append(char)
            resultLineDic = {}
            for key, line in lineDic.items():
                text = ''
                temp = sorted(line['element'], key=lambda x:x['x0'], reverse=False)
                leftBound = temp[0]['x0']  # 最左字符x0
                rightBound = temp[-1]['x1']  # 最右字符x1
                lowerBound = None
                upperBound = None
                for char in line['element']:
                    text += char['text']
                    lowerBound = char['y0'] if lowerBound == None or char['y0'] < lowerBound else lowerBound
                    upperBound = char['y1'] if upperBound == None or char['y1'] > upperBound else upperBound
                text = re.sub(r'[\s]+', '', text)
                if len(text) != 0:
                    lineDic[key]['content'] = text
                    lineDic[key]['x0'] = leftBound
                    lineDic[key]['x1'] = rightBound
                    lineDic[key]['y0'] = lowerBound
                    lineDic[key]['y1'] = upperBound
                    resultLineDic[key] = lineDic[key]
            pdfDic[pageNo]['lineDic'] = resultLineDic
            break

    return pdfDic
def getKeywordRegion(keyword, pdfDic):
    '''

    :param keyword: 待定位的关键词
    :param pdfDic: analyzePDF_step1_lineInfo返回的字典
    :return:
    '''
    location = {}
    for pageNo, lineDic in pdfDic.items():
        for _, line in lineDic['lineDic'].items():
            if keyword == line['content']:
                location['pageNo'] = pageNo
                location['x0'] = float(line['x0'])
                location['x1'] = float(line['x1'])
                location['y0'] = float(line['y0'])
                location['y1'] = float(line['y1'])
    return location

In [232]:
pdfDic = analyzePDF_step1_lineInfo(feePDF)

In [236]:
getKeywordRegion('目录', pdfDic)

{}

In [38]:
str_num = "二十一"
# 一、定义转换字典
num_dict = {"一": "1", "二": "2", "三": "3", "四": "4", "五": "5", "六": "6", "七": "7", "八": "8", "九": "9", "十": ""}
# 将中文的数字替换成阿拉伯数字
# 针对不同的情况进行判断和字典的更改
if str_num[0] == "十" and len(str_num) > 1:
    num_dict["十"] = "1"
if str_num[0] == "十" and len(str_num) == 1:
    num_dict["十"] = "10"
if str_num[1] == "十" and len(str_num) == 2:
    num_dict["十"] = "0"
num = ""
# 遍历字典并对中文数字进行替换
for str in str_num:
    for key in num_dict:
        if key == str:
            num += num_dict[key]
            break
print(num)


21


# 2. 用lxml解析html文本

In [34]:
import lxml
from lxml import etree

# 3. 前两种方法结合

In [722]:
b

In [720]:
b = pdf.pages[0]
b.bbox

(Decimal('0'), Decimal('0'), Decimal('595.200'), Decimal('841.920'))

In [723]:
c = b.crop((Decimal('0'), Decimal('0'), Decimal('295.200'), Decimal('441.920')))

In [725]:
c.extract_text()

' \n阅  读 \n\uf043\n平安小福星终身寿险产品提供身故保障 \n\uf043\n为了帮助您更好地了解产品，我们先介绍几个保\n\uf076  被保险人就是受保险合同保障的人。 \n\uf076  投保人就是购买保险并交纳保险费的人。 \n\uf076  受益人就是发生保险事故后领取保险金的人\n\uf076  保险人就是保险公司。 \n\uf043\n下面我们举例说明本产品提供哪些保障 \n例子：王先生为儿子小王（0 周岁）投保平安小\n51万元，指定身故保险金受益人为王先生本人。\n险（简称小福星重疾）。 \n本例中王先生为投保人及受益人，小王为被保险\n保险金  领取人  给付金额\n身故保险金  王先生  51万元\n以上举例仅供您更好地理解产品之用，您所购买\n\uf043 \n条款目录 \n '